# Modulo 5: Inferencia estadística y modelo de regresión lineal

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Preparación de la data

In [ ]:
df_data = pd.read_excel(r'C:\Carrera de Datos\Data Scientist\Diplomado Python EAN\Archivos\final_df.xlsx')

In [ ]:
df_data.describe()

In [ ]:
df_data.columns

In [ ]:
df_data=df_data.rename(columns={'Nivel_Educativo':'Nivel Educativo','tipo_vivienda':'Tipo Vivienda'})

In [ ]:
df_data.columns

In [ ]:
df_data.shape

# Modelo de regresión lineal simple

Vamos a estimar un modelo en donde se analice la distribución de la variable Ingreso en función del Nivel educativo

In [ ]:
import statsmodels.api as sm

In [ ]:
f, axs = plt.subplots(1, 3)
sns.boxplot(data=df_data["Ingresos"], ax=axs[0])

sns.boxplot(data=df_data["Nivel Educativo"], ax=axs[1])

sns.boxplot(data=df_data["Bienes"])

In [ ]:
q1 = df_data.Ingresos.quantile(0.25)
q3 = df_data.Ingresos.quantile(0.75)
RIQ = q3-q1

In [ ]:
#f, axs = plt.subplots(1, 2)
sns.boxplot(data=df_data["Ingresos"])
plt.axhline(min(df_data.Ingresos), color='g')
plt.axhline(q3+1.5*RIQ, color='b')
plt.axhline(q3+3*RIQ, color='r')
sns.boxplot(data=df_data["Nivel Educativo"])

###  Gráfico dinámico

In [ ]:
def plot_function(x_mill = 9000000,y=50000, bins = 10, color='red'):
    binwidth = (max(df_data.Ingresos) - min(df_data.Ingresos))/ bins
    plt.hist(df_data.Ingresos, 
             bins=np.arange(min(df_data.Ingresos), max(df_data.Ingresos) + binwidth, binwidth),
             color=color)
    plt.xlim(0,x_mill)
    plt.ylim(0,y)
    plt.show()

In [ ]:
from ipywidgets import interact 
interact(plot_function,
         x_mill = (0, 9000000, 100000),
         y = (0, 50000, 1000),
         bins = (1, 1000, 1),
         color=['red', 'yellow', 'blue','gray','green','white','black'])
None

### Selección de una muestra para correr el modelo

Inicialmente vamos a correr el modelo con datos que se encuentren dentro del recorrido intercuantil y además vamos a tomar sólo una muestra al azar de 100 de ellos para ver mejor las gráficas.

In [ ]:
df_data.head()

In [ ]:
#from numpy.core.fromnumeric import repeat
#indices = df_data[(df_data.Ingresos<q3) & (df_data.Ingresos>q1)].index
#display(indices)
#muestra=np.random.choice(indices,100,replace=False)
#display(muestra)
#df_data=df_data.loc[muestra]

#### Eliminación de datos atípicos anormales

In [ ]:
df_data = df_data[df_data.Ingresos<q3+1.5*RIQ]
df_data.shape

## Diagrama de dispersión de Ingresos y Nivel educativo

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

df_data.plot(
    x    = 'Bienes',
    y    = 'Ingresos',
    c    = 'yellow',
    kind = "scatter",
    ax   = ax
)
ax.set_title('Ingresos vs Bienes');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

df_data.plot(
    x    = 'Nivel Educativo',
    y    = 'Ingresos',
    c    = 'green',
    kind = "scatter",
    ax   = ax
)
ax.set_title('Ingresos vs NIvel educativo');

## Correlación lineal entre las variables

In [ ]:
from scipy.stats import pearsonr
corr_test = pearsonr(x = df_data['Nivel Educativo'], y =  df_data['Ingresos'])
print("Coeficiente de correlación de Pearson: ", corr_test[0])
print("P-value: ", corr_test[1])

In [ ]:
from scipy.stats import pearsonr
corr_test = pearsonr(x = df_data['Bienes'], y =  df_data['Ingresos'])
print("Coeficiente de correlación de Pearson: ", corr_test[0])
print("P-value: ", corr_test[1])

## Estimación del modelo

Selección de variables independientes

In [ ]:
inde=['Nivel Educativo','Bienes']
X=df_data[inde]
X.head(10)

Agregar la constante

In [ ]:
X= sm.add_constant(X)

In [ ]:
X

Selección de variable dependiente

In [ ]:
y=df_data['Ingresos']
y

Separación de registros de entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 100)

In [ ]:
display(X_train.head(2))
display(y_train.head(2))
display(X_test.head(2))
display(y_test.head(2))

In [ ]:
X_train = sm.add_constant(X_train)
X_test= sm.add_constant(X_test)

In [ ]:
model = sm.OLS(y_train,X_train)
results = model.fit()

In [ ]:
print(results.summary())

In [ ]:
results.params

In [ ]:
display(X_test.head(3))
display(y_test.head(3))

## Intervalos de confianza

In [ ]:
results.conf_int(alpha=0.05) #Confianza del 95%

# Predicciones

In [ ]:
display(X_test.head())

In [ ]:
display(y_test.head())

In [ ]:
results.predict(X_test).head()

## Intervalos de confianza predicciones

In [ ]:
Pred_test=results.get_prediction(X_test).summary_frame()#.sort_values("mean")

In [ ]:
Pred_test

In [ ]:
Pred_test["X"]=X_test['Nivel Educativo']

In [ ]:
Pred_test["Nivel educativo"]=X_test['Nivel Educativo']
Pred_test["Bienes"]=X_test['Bienes']

Pred_test["Observado"]= y_test

In [ ]:
Pred_test

In [ ]:
Pred_test=Pred_test.sort_values("mean")

In [ ]:
Pred_test

## Gráfica de la regresión lineal

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

ax.scatter(Pred_test["X"], Pred_test.Observado, marker='o', color = "blue")
ax.plot(Pred_test["Nivel educativo"], Pred_test["mean"], linestyle='-', label="OLS",color="red")
ax.plot(Pred_test["Nivel educativo"], Pred_test["mean_ci_lower"], linestyle='--', color='red', label="95% CI media")
ax.plot(Pred_test["Nivel educativo"], Pred_test["mean_ci_upper"], linestyle='--', color='red')
ax.fill_between(Pred_test["Nivel educativo"], Pred_test["mean_ci_lower"], Pred_test["mean_ci_upper"], alpha=0.1)
ax.plot(Pred_test["Nivel educativo"], Pred_test["obs_ci_lower"], linestyle='--', color='blue', label="95% CI Observaciones")
ax.plot(Pred_test["Nivel educativo"], Pred_test["obs_ci_upper"], linestyle='--', color='blue')


ax.legend();

## Pruebas de bondad de ajuste

In [ ]:
print('coefficient of determination:', results.rsquared)
print('adjusted coefficient of determination:', results.rsquared_adj)
print('regression coefficients:', results.params)

In [ ]:
from sklearn.metrics import mean_squared_error
predicciones = results.predict(X_test)
rmse = mean_squared_error(
        y_true  = y_test,
        y_pred  = predicciones,
        squared = False
       )
print(f"El error (rmse) de test es: {rmse}")

## Diagnóstico de los residuos

Residuos de entrenamiento

In [ ]:
residuos_train = y_train - results.predict(X_train)

In [ ]:
residuos_train.describe()

Residuos de prueba

In [ ]:
residuos_test = Pred_test["Observado"] - Pred_test["mean"]

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(9, 8))

axes[0, 0].scatter(Pred_test["Observado"],Pred_test["mean"], edgecolors=(0, 0, 0), alpha = 0.4)
axes[0, 0].plot([Pred_test["Observado"].min(), Pred_test["Observado"].max()],
                [Pred_test["Observado"].min(), Pred_test["Observado"].max()],
                'k--', color = 'black', lw=2)
axes[0, 0].set_title('Valor predicho vs valor real', fontsize = 10, fontweight = "bold")
axes[0, 0].set_xlabel('Real')
axes[0, 0].set_ylabel('Predicción')
axes[0, 0].tick_params(labelsize = 7)

axes[0, 1].scatter(list(range(len(residuos_test))), residuos_test,
                   edgecolors=(0, 0, 0), alpha = 0.4)
axes[0, 1].axhline(y = 0, linestyle = '--', color = 'black', lw=2)
axes[0, 1].set_title('Residuos del modelo', fontsize = 10, fontweight = "bold")
axes[0, 1].set_xlabel('id')
axes[0, 1].set_ylabel('Residuo')
axes[0, 1].tick_params(labelsize = 7)

sns.histplot(
    data    = residuos_test,
    stat    = "density",
    kde     = True,
    line_kws= {'linewidth': 1},
    color   = "firebrick",
    alpha   = 0.3,
    ax      = axes[1, 0]
)

axes[1, 0].set_title('Distribución residuos del modelo', fontsize = 10,
                     fontweight = "bold")
axes[1, 0].set_xlabel("Residuo")
axes[1, 0].tick_params(labelsize = 7)


sm.qqplot(
    residuos_test,
    fit   = True,
    line  = 'q',
    ax    = axes[1, 1], 
    color = 'firebrick',
    alpha = 0.4,
    lw    = 2
)
axes[1, 1].set_title('Q-Q residuos del modelo', fontsize = 10, fontweight = "bold")
axes[1, 1].tick_params(labelsize = 7)

axes[2, 0].scatter(Pred_test["mean"], residuos_test,
                   edgecolors=(0, 0, 0), alpha = 0.4)
axes[2, 0].axhline(y = 0, linestyle = '--', color = 'black', lw=2)
axes[2, 0].set_title('Residuos del modelo vs predicción', fontsize = 10, fontweight = "bold")
axes[2, 0].set_xlabel('Predicción')
axes[2, 0].set_ylabel('Residuo')
axes[2, 0].tick_params(labelsize = 7)

# Se eliminan los axes vacíos
fig.delaxes(axes[2,1])

fig.tight_layout()
plt.subplots_adjust(top=0.9)
fig.suptitle('Diagnóstico residuos', fontsize = 12, fontweight = "bold");

## Test de Normalidad

In [ ]:
from scipy import stats
shapiro_test = stats.shapiro(residuos_test)
shapiro_test